In [1]:
import random
import json
from faker import Faker

# Inicializa o Faker para dados em Português-Brasil
fake = Faker("pt_BR")

# 1. Constantes do seu template
LABEL = "carteira_oab"
EXTRACTION_SCHEMA = {
    "nome": "Nome do profissional, normalmente no canto superior esquerdo da imagem",
    "inscricao": "Número de inscrição do profissional",
    "seccional": "Seccional do profissional",
    "subsecao": "Subseção à qual o profissional faz parte",
    "categoria": "Categoria, pode ser ADVOGADO, ADVOGADA, SUPLEMENTAR, ESTAGIARIO, ESTAGIARIA",
    "endereco_profissional": "Endereço do profissional",
    "telefone_profissional": "Telefone do profissional",
    "situacao": "Situação do profissional, normalmente no canto inferior direito.",
}

# 2. Template de texto OCR "falsificado"
# Usamos {chaves} para injetar os dados
OCR_TEMPLATE = """
ORDEM DOS ADVOGADOS DO BRASIL
CONSELHO SECCIONAL DE {seccional}

Nome
{nome}

Inscrição: {inscricao}       Categoria: {categoria}

Subseção: {subsecao}

Endereço Profissional: {endereco_profissional}

Telefone Profissional: {telefone_profissional}

Situação
{situacao}

Validade: {validade}
"""


# 3. Funções para gerar dados
def generate_canonical_record():
    """Gera um dicionário de dados 'corretos'."""
    seccional = fake.state_abbr()
    return {
        "nome": fake.name(),
        "inscricao": str(random.randint(100000, 999999)),
        "seccional": seccional,
        "subsecao": f"{random.randint(1, 20)} - {fake.city()}",
        "categoria": random.choice(
            ["ADVOGADO", "ADVOGADA", "SUPLEMENTAR", "ESTAGIARIO"]
        ),
        "endereco_profissional": fake.address().replace("\n", ", "),
        "telefone_profissional": fake.phone_number(),
        "situacao": "REGULAR",
        "validade": fake.date_of_birth(minimum_age=1, maximum_age=5).strftime(
            "%d/%m/%Y"
        ),
    }


def generate_wrong_data(field, canonical_value):
    """Gera dados 'errados' (diferentes do canônico)."""
    wrong_data = ""
    # Garante que o dado "errado" seja diferente do "correto"
    while not wrong_data or wrong_data == canonical_value:
        if field == "nome":
            wrong_data = fake.name()
        elif field == "inscricao":
            wrong_data = str(random.randint(100000, 999999))
        elif field == "seccional":
            wrong_data = fake.state_abbr()
        elif field == "subsecao":
            wrong_data = f"{random.randint(1, 20)} - {fake.city()}"
        elif field == "categoria":
            wrong_data = random.choice(
                ["ADVOGADO", "ADVOGADA", "SUPLEMENTAR", "ESTAGIARIO"]
            )
        elif field == "endereco_profissional":
            wrong_data = fake.address().replace("\n", ", ")
        elif field == "telefone_profissional":
            wrong_data = fake.phone_number()
        elif field == "situacao":
            wrong_data = random.choice(["SUSPENSO", "CANCELADO"])
        else:
            wrong_data = "DADO ERRADO"
    return wrong_data


# 4. Função principal para gerar uma amostra
def generate_sample():
    """Gera um único item do dataset com variações."""

    # Começa com um registro "perfeito"
    canonical_record = generate_canonical_record()

    # Dicionários para construir a amostra
    template_data = canonical_record.copy()  # Dados que irão para o template de texto
    expected_answer = {}  # O que a extração DEVE encontrar

    fields_to_process = list(EXTRACTION_SCHEMA.keys())

    for field in fields_to_process:
        state = random.choice(["correct", "wrong", "empty"])

        if state == "correct":
            # A informação correta está no texto.
            # O 'expected_answer' deve ser a informação correta.
            expected_answer[field] = canonical_record[field]
            template_data[field] = canonical_record[field]

        elif state == "wrong":
            # Uma informação "errada" é colocada no texto.
            # O 'expected_answer' deve ser essa informação "errada".
            wrong_data = generate_wrong_data(field, canonical_record[field])
            expected_answer[field] = wrong_data
            template_data[field] = wrong_data  # Injeta o dado errado no template

        elif state == "empty":
            # A informação está ausente no texto.
            # O 'expected_answer' deve ser None (ou string vazia).
            expected_answer[field] = None

            # Simula um OCR que ou deixa em branco ou omite a linha
            if random.random() < 0.5:
                template_data[field] = ""  # Deixa o campo em branco no template
            else:
                # Simula um OCR que pode adicionar "ruído"
                template_data[field] = random.choice([" ", "---", "N/D"])

    # Formata o texto final do OCR
    # Usamos .get(k, '') para lidar com campos extras no template (como 'validade')
    # que não estão no schema de extração.
    final_ocr_text = OCR_TEMPLATE.format_map(
        {
            k: template_data.get(k, canonical_record.get(k, ""))
            for k in (list(template_data.keys()) + list(canonical_record.keys()))
        }
    )

    return {
        "label": LABEL,
        "extraction_schema": EXTRACTION_SCHEMA,
        "ocr_text": final_ocr_text,
        "expected_answer": expected_answer,
    }


# --- Execução ---
if __name__ == "__main__":
    NUM_SAMPLES = 1  # Mude aqui para gerar mais amostras

    dataset = [generate_sample() for _ in range(NUM_SAMPLES)]

    # Imprime o dataset gerado em formato JSON legível
    print(json.dumps(dataset, indent=2, ensure_ascii=False))

[
  {
    "label": "carteira_oab",
    "extraction_schema": {
      "nome": "Nome do profissional, normalmente no canto superior esquerdo da imagem",
      "inscricao": "Número de inscrição do profissional",
      "seccional": "Seccional do profissional",
      "subsecao": "Subseção à qual o profissional faz parte",
      "categoria": "Categoria, pode ser ADVOGADO, ADVOGADA, SUPLEMENTAR, ESTAGIARIO, ESTAGIARIA",
      "endereco_profissional": "Endereço do profissional",
      "telefone_profissional": "Telefone do profissional",
      "situacao": "Situação do profissional, normalmente no canto inferior direito."
    },
    "ocr_text": "\nORDEM DOS ADVOGADOS DO BRASIL\nCONSELHO SECCIONAL DE \n\nNome\nKevin Silveira\n\nInscrição: 770961       Categoria: ADVOGADO\n\nSubseção: 16 - Aragão de Minas\n\nEndereço Profissional: Travessa Nunes, 55, Vila Havaí, 77968-607 Garcia da Praia / SC\n\nTelefone Profissional: 71 0252-8027\n\nSituação\nSUSPENSO\n\nValidade: 10/03/2022\n",
    "expected_answ

In [2]:
fake.city()

'da Cunha do Campo'

In [3]:
import random
import json
from faker import Faker
import string

# Inicializa o Faker para dados em Português-Brasil
fake = Faker("pt_BR")


def seed_random(seed_value):
    random.seed(seed_value)
    fake.seed_instance(seed_value)


# 1. Constantes do seu template
LABEL = "carteira_oab"
EXTRACTION_SCHEMA = {
    "nome": "Nome do profissional, normalmente no canto superior esquerdo da imagem",
    "inscricao": "Número de inscrição do profissional",
    "seccional": "Seccional do profissional",
    "subsecao": "Subseção à qual o profissional faz parte",
    "categoria": "Categoria, pode ser ADVOGADO, ADVOGADA, SUPLEMENTAR, ESTAGIARIO, ESTAGIARIA",
    "endereco_profissional": "Endereço do profissional",
    "telefone_profissional": "Telefone do profissional",
    "situacao": "Situação do profissional, normalmente no canto inferior direito.",
}


# 2. Funções para gerar dados
def generate_canonical_record():
    """Gera um dicionário de dados 'corretos'."""
    return {
        "nome": fake.name(),
        "inscricao": fake.rg(),
        "seccional": fake.state_abbr(),
        "subsecao": f"{fake.city()} - {fake.state()}",
        "categoria": random.choice(
            ["ADVOGADO", "ADVOGADA", "SUPLEMENTAR", "ESTAGIARIO"]
        ),
        "endereco_profissional": fake.address().replace("\n", ", "),
        "telefone_profissional": fake.phone_number(),
        "situacao": "Regular",
    }


def generate_wrong_data(field, canonical_value):
    """Gera dados 'errados' de forma bagunçada."""
    wrong_data = canonical_value

    while wrong_data == canonical_value:
        if field == "nome":
            # Adiciona números e símbolos ao nome
            wrong_data = (
                canonical_value
                + " "
                + "".join(random.choices(string.ascii_letters + string.digits, k=3))
            )
        elif field == "inscricao":
            # Número de 1 ou 2 dígitos
            wrong_data = str(random.randint(0, 99))
        elif field == "seccional":
            # Mais de 2 letras ou números
            wrong_data = "".join(
                random.choices(
                    string.ascii_letters + string.digits, k=random.randint(3, 5)
                )
            )
        elif field == "subsecao":
            # Troca cidade e estado, adiciona número aleatório
            wrong_data = f"{fake.city()} {random.randint(1, 99)} - {fake.state()}"
        elif field == "categoria":
            # Categoria aleatória bagunçada
            wrong_data = random.choice(["ADV", "EST", "SUP", "ADVOGADA123", ""])
        elif field == "endereco_profissional":
            # Adiciona símbolos e quebras de linha
            wrong_data = (
                canonical_value.replace(",", ";") + " #" + str(random.randint(1, 100))
            )
        elif field == "telefone_profissional":
            # Formato errado, letras misturadas
            wrong_data = "".join(
                random.choices(string.digits + string.ascii_letters, k=10)
            )
        elif field == "situacao":
            wrong_data = random.choice(["SUSPENSO", "CANCELADO", "BLOQUEADO", ""])
        else:
            wrong_data = "DADO ERRADO " + str(random.randint(0, 999))

    return wrong_data


def fuzz_text(text):
    """Aplica ruído em nível de caractere a uma string."""
    if not text:
        return text
    text = str(text)
    # Mapa de ruído (15% de chance de trocar)
    noise_map = {
        "O": "0",
        "o": "0",
        "l": "1",
        "i": "1",
        "S": "5",
        "s": "5",
        "B": "8",
        "A": "4",
    }
    fuzzed = ""
    for char in text:
        # if char in noise_map and random.random() < 0.15:
        #     fuzzed += noise_map[char]
        # elif (
        if char == " " and random.random() < 0.1:  # 10% de chance de remover um espaço
            pass
        else:
            fuzzed += char

    return fuzzed


# 3. Função principal para gerar uma amostra "suja"
def generate_sample():
    """Gera um único item do dataset com OCR "sujo"."""

    canonical_record = generate_canonical_record()
    ocr_chunks = []  # Pedaços de texto que formarão o OCR
    expected_answer = {}  # O que a extração DEVE encontrar

    # Adiciona um lixo no início, às vezes
    if random.random() < 0.3:
        ocr_chunks.append(
            random.choice(["ORDEM DOS ADV0GAD0S", "SCAN DOC 001", fake.company()])
        )

    for field in EXTRACTION_SCHEMA.keys():
        # 1. Decidir o estado do VALOR
        state = random.choice(
            [
                "correct",
                "correct",
                "correct",
                "correct",
                "correct",
                "wrong",
                "wrong",
                "omitted",
                "omitted",
                "omitted",
            ]
        )  # , "placeholder"])

        value = None
        expected_value = None

        if state == "correct":
            value = canonical_record[field]
            expected_value = value
        elif state == "wrong":
            value = generate_wrong_data(field, canonical_record[field])
            expected_value = value
        # elif state == "placeholder":
        #     value = random.choice(["N/D", "---", "???", "NA", "ILEGIVEL"])
        #     expected_value = value  # O modelo deve ser capaz de extrair "N/D"
        elif state == "omitted":
            expected_value = None
            # Mesmo omitido, o rótulo pode aparecer
            if random.random() < 0.3:
                ocr_chunks.append(fuzz_text(field.replace("_", " ").title()))
            continue  # Pula para o próximo campo

        expected_answer[field] = expected_value

        # 2. Decidir o estado do RÓTULO (Label)
        label_text_base = field.replace("_", " ").title()
        label_state = random.choice(
            [
                "full",
                "full",
                "full",
                "full",
                "full",
                "mixed",
                "mixed",
                "mixed",
                "partial",
                "omitted",
            ]
        )

        fuzzed_value = fuzz_text(value)

        # 3. Adicionar os pedaços de texto
        if label_state == "full":
            # Ex: "Nome" "Maria Luiza"
            ocr_chunks.append(fuzz_text(label_text_base))
            ocr_chunks.append(fuzzed_value)

        elif label_state == "partial":
            # Ex: "Inscr:" "123456"
            ocr_chunks.append(fuzz_text(label_text_base.split()[0]))
            ocr_chunks.append(fuzzed_value)

        elif label_state == "omitted":
            # Ex: "Maria Luiza" (sem rótulo)
            ocr_chunks.append(fuzzed_value)

        elif label_state == "mixed":
            # Ex: "SITUAÇÃO REGULAR" ou "NomeMaria Luiza"
            if random.random() < 0.5:
                # "SITUAÇÃO REGULAR"
                ocr_chunks.append(fuzz_text(f"{label_text_base.split()[0]} {value}"))
            else:
                # "NomeMaria Luiza"
                ocr_chunks.append(fuzz_text(f"{label_text_base.split()[0]}{value}"))

    # 4. Montar o ocr_text final com separadores "sujos"
    final_ocr_text = ""
    for chunk in ocr_chunks:
        final_ocr_text += chunk
        # O separador é o segredo para a "sujeira"
        separator = random.choice(
            [
                "\n",
                "\n",
                "\n",
                "\n",
                "\n",
                "\n",
                "\n",
                "\n",
                " ",
                " ",
                " ",
                "",
                "",
                "   ",
                "   ",
                "\t",
                "\t",
            ]
        )
        final_ocr_text += separator

    return {
        "label": LABEL,
        "extraction_schema": EXTRACTION_SCHEMA,
        "ocr_text": final_ocr_text,  # Remove espaços no início/fim
        "expected_answer": expected_answer,
    }


# --- Execução ---
if __name__ == "__main__":
    seed_random(42)  # Para resultados reprodutíveis

    NUM_SAMPLES = 5  # Mude aqui para gerar mais amostras

    dataset = [generate_sample() for _ in range(NUM_SAMPLES)]

    # Imprime o dataset gerado em formato JSON legível
    print(json.dumps(dataset, indent=2, ensure_ascii=False))

[
  {
    "label": "carteira_oab",
    "extraction_schema": {
      "nome": "Nome do profissional, normalmente no canto superior esquerdo da imagem",
      "inscricao": "Número de inscrição do profissional",
      "seccional": "Seccional do profissional",
      "subsecao": "Subseção à qual o profissional faz parte",
      "categoria": "Categoria, pode ser ADVOGADO, ADVOGADA, SUPLEMENTAR, ESTAGIARIO, ESTAGIARIA",
      "endereco_profissional": "Endereço do profissional",
      "telefone_profissional": "Telefone do profissional",
      "situacao": "Situação do profissional, normalmente no canto inferior direito."
    },
    "ocr_text": "SCAN DOC 001   Nome Brenda Alves Inscricao\n431608258\nAL Subsecao\nGarcia5 - Amapá\nCategoriaSituacao\nRegular",
    "expected_answer": {
      "nome": "Brenda Alves",
      "inscricao": "431608258",
      "seccional": "AL",
      "subsecao": "Garcia 5 - Amapá",
      "situacao": "Regular"
    }
  },
  {
    "label": "carteira_oab",
    "extraction_schem

In [4]:
print(dataset[3]["ocr_text"])

NomeKaique Porto Inscricao
56
Seccional
Subsecao
Camargo - Mato Grosso doSul   Categoria
ADVOGADA   Endereco Profissional
Feira Nunes,2, Vila Ouro Minas, 18227-824 Cunha da Prata / SP Telefone+55(081) 4657-8713   Situacao
Regular

